In [35]:
import sys
sys.path.append('../..')

import numpy as np
import pandas as pd
import gru4rec
import evaluation

PATH_TO_TRAIN = 'processedrsc15_train_full.txt'
PATH_TO_TEST = 'processedrsc15_test.txt'
pd.options.display.max_rows = 10

In [2]:
data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId':np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId':np.int64})

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31637239 entries, 0 to 31637238
Data columns (total 3 columns):
SessionId    int64
ItemId       int64
Time         float64
dtypes: float64(1), int64(2)
memory usage: 724.1 MB


In [4]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71222 entries, 0 to 71221
Data columns (total 3 columns):
SessionId    71222 non-null int64
ItemId       71222 non-null int64
Time         71222 non-null float64
dtypes: float64(1), int64(2)
memory usage: 1.6 MB


In [5]:
gru = gru4rec.GRU4Rec(loss='top1-max',final_act='softmax',hidden_act='relu',
                      layers=[100],batch_size=50, dropout_p_hidden=0.5,
                      learning_rate=0.01,momentum=0.0,time_sort=False)

In [6]:
%time gru.fit(data)

Epoch0	loss: 0.994812
Epoch1	loss: 0.975859
Epoch2	loss: 0.957853
Epoch3	loss: 0.945507
Epoch4	loss: 0.936542
Epoch5	loss: 0.929307
Epoch6	loss: 0.923198
Epoch7	loss: 0.918094
Epoch8	loss: 0.913777
Epoch9	loss: 0.910165
CPU times: user 2h 49min 39s, sys: 1min 20s, total: 2h 51min
Wall time: 2h 51min 8s


In [7]:
res = evaluation.evaluate_sessions_batch(gru,valid,None)

Measuring Recall@19 and MRR@19


In [8]:
res

(0.39414648109055778, 0.16676975545085579)

In [ ]:
gru = gru4rec.GRU4Rec(loss='bpr', final_act='tanh', hidden_act='tanh', 
                      layers=[100], batch_size=50, dropout_p_hidden=0.5, 
                      learning_rate=0.01, momentum=0.0, time_sort=False)

In [ ]:
%time gru.fit(data)

In [ ]:
res = evaluation.evaluate_sessions_batch(gru, valid, None)
print('Recall@20: {}'.format(res[0]))
print('MRR@20: {}'.format(res[1]))

In [ ]:
gru = gru4rec.GRU4Rec(loss='bpr-max-0.5', final_act='linear', hidden_act='tanh', 
                      layers=[100], batch_size=32, dropout_p_hidden=0.0, learning_rate=0.2, 
                      momentum=0.5, n_sample=2048, sample_alpha=0, time_sort=True)

In [ ]:
%time gru.fit(data)

In [ ]:
len(res)

In [9]:
ls

dataset-README.txt              reddit_03_17_posts.csv
preprocess.py                   run_rsc15.py
processed/                      test_model-2.ipynb
processedrsc15_test.txt         test_model-3.ipynb
processedrsc15_train_full.txt   test_model.ipynb
processedrsc15_train_tr.txt     yoochoose-buys.dat
processedrsc15_train_valid.txt  yoochoose-clicks.dat
reddit_01_17_posts.csv          yoochoose-data.7z
reddit_02_17_posts.csv          yoochoose-test.dat


In [13]:
dR = pd.read_csv('reddit_01_17_posts.csv',header=None)

In [19]:
dR.columns=['Time','UserName','Topic','SubReddit']

#### Drop all the nan's and drop the topic

In [31]:
dR.dropna(inplace=True)
dR.drop(['Topic'],axis=1,inplace=True)

In [36]:
dR

,Time,UserName,SubReddit
10,1483304874,farbenwvnder,de
11,1484153578,Jordywp,de
12,1484252123,YaWishYouHadThatName,de
13,1484254069,ProgNose,de
14,1484292642,vinokess,de
...,...,...,...
9218508,1484587034,MatTEs_Carter,rocket_league_trading
9218509,1484568550,vilcatiwordjob,shittycomputersupport
9218510,1484493772,Jacobamvin,thescienceofdeduction
9218511,1484275294,qbang05,upliftingnewsforwomen


#### Group by UserName

In [27]:
grp=dR.groupby('UserName')

In [40]:
dir(grp)

['SubReddit',
 'Time',
 'UserName',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_add_numeric_operations',
 '_agg_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_block_agg_axis',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_decide_output_index',
 '_def_str',
 '_define_paths',
 '_dir_additions',
 '_dir_deletions',
 '_get_da

In [46]:
dR['Count'] = 0

In [56]:
grp.all()

,Count,SubReddit,Time
UserName,,,
------------------GL,False,True,True
--------------10,False,True,True
----------_-_-,False,True,True
---------45434--,False,True,True
--------_-------,False,True,True
...,...,...,...
zzzzzzzzdfghjkl,False,True,True
zzzzzzzzlll,False,True,True
zzzzzzzzz_z,False,True,True
